In [ ]:
# >>>> MULTIPLOS ATIVOS <<<<

from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot
import pytz

agora = datetime.now()
print(f'Buscando dados...{agora}')
# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'


if not mt5.initialize(login=1092947504, server="ClearInvestimentos-DEMO", password="Joh0516"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

def run():    
    symbols = ["WDOK23","WINM23","PETR4","VALE3","ITUB4"]
    #symbol = "WDOK23"

    for symbol in symbols: 
        print(f'\nAtivo: {symbol} - {datetime.now()}')
        timezone = pytz.timezone("Etc/UTC")
        #utc_from = datetime(2021, 12, 2, tzinfo=timezone)
        rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 98)
        mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_W1, 0, 210)
        rates_frame = pd.DataFrame(rates)
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        resumo = rates_frame[['time','open','close','spread']]
        resumoCloseGeral = resumo[['time','close']]
        resumoCloseArmt = resumoCloseGeral.tail(3)

        NoveMME = resumoCloseGeral['close'].ewm(span=9).mean() 
        TresMMA = resumoCloseGeral['close'].ewm(span=3).mean()
        TresMMATeste = resumoCloseArmt['close'].mean()

        resumoCloseGeral.insert(loc=2,column='MME 9',value=NoveMME)
        resumoCloseGeral.insert(loc=3,column='MME 3',value=TresMMA)
        resumoCloseGeral.insert(loc=4,column='MMA 3 Teste',value=TresMMATeste)

        def enviaAlertaCompra():
            bot = telepot.Bot('5026686955:AAHvm0rJOf-_nSCi8sOHYVMhY8zPCBEd73k')
            bot.sendMessage(984798692, f'ESTRATÉGIA 9.1 EM ANDAMENTO: >> {symbol}/COMPRA <<')
            print('Dados encontrados e enviados via Telegram'.upper())

        def enviaAlertaVenda():
            bot = telepot.Bot('5026686955:AAHvm0rJOf-_nSCi8sOHYVMhY8zPCBEd73k')
            bot.sendMessage(984798692, f'ESTRATÉGIA 9.1 EM ANDAMENTO: >> {symbol}/VENDA <<')
            print('Dados encontrados e enviados via Telegram'.upper())

        # MONITORAMENTO
        pd.options.mode.chained_assignment = None
        resumoCloseGeral['flag'] = ''
        resumoCloseGeral['sinal'] = ''

        for i in range (1, len(resumo)):
            if resumoCloseGeral['MME 9'][i] < resumoCloseGeral['MME 3'][i]:
                resumoCloseGeral['flag'][i] = 'COMPRA'

            if resumoCloseGeral['MME 9'][i] > resumoCloseGeral['MME 3'][i]:
                resumoCloseGeral['flag'][i] = 'VENDA'


        for x in range(1,len(resumoCloseGeral)):
            if resumoCloseGeral['flag'][x] == resumoCloseGeral['flag'][x-1]:
                resumoCloseGeral['sinal'][x] = ''
            else:
                resumoCloseGeral['sinal'][x] = 'sinal'

        if resumoCloseGeral['sinal'].iloc[-1] == 'sinal':
            if resumoCloseGeral['flag'] == 'COMPRA':
                enviaAlertaCompra()
                print('Compra')
            else:
                resumoCloseGeral['flag'][i] = 'VENDA'
                print('Venda')
                enviaAlertaVenda()

        resumoCloseGeral.tail(5)
        
while True:
    run()
    time.sleep(300)

In [ ]:
from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot
import pytz

agora = datetime.now()
print(f'Buscando dados...{agora}')
# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'


if not mt5.initialize(login=1092947504, server="ClearInvestimentos-DEMO", password="Joh0516"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

#symbols = ["WDOK23","WINM23","PETR4"]
symbol = "WDOK23"

#for symbol in symbols:
print(f'\nAtivo: {symbol}')
timezone = pytz.timezone("Etc/UTC")
#utc_from = datetime(2021, 12, 2, tzinfo=timezone)
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 98)
mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_W1, 0, 210)
rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
resumo = rates_frame[['time','open','close','spread']]
resumoCloseGeral = resumo[['time','close']]
#resumoCloseExp = resumoCloseGeral.tail(9)
resumoCloseArmt = resumoCloseGeral.tail(3)

NoveMME = resumoCloseGeral['close'].ewm(span=9).mean() 
TresMMA = resumoCloseGeral['close'].ewm(span=3).mean()
TresMMATeste = resumoCloseArmt['close'].mean()

resumoCloseGeral.insert(loc=2,column='MME 9',value=NoveMME)
resumoCloseGeral.insert(loc=3,column='MME 3',value=TresMMA)
resumoCloseGeral.insert(loc=4,column='MMA 3 Teste',value=TresMMATeste)

def enviaAlertaCompra():
    bot = telepot.Bot('5026686955:AAHvm0rJOf-_nSCi8sOHYVMhY8zPCBEd73k')
    bot.sendMessage(984798692, f'ESTRATÉGIA 9.1 EM ANDAMENTO: >> {symbol}/COMPRA <<')
    print('Dados encontrados e enviados via Telegram'.upper())

def enviaAlertaVenda():
    bot = telepot.Bot('5026686955:AAHvm0rJOf-_nSCi8sOHYVMhY8zPCBEd73k')
    bot.sendMessage(984798692, f'ESTRATÉGIA 9.1 EM ANDAMENTO: >> {symbol}/VENDA <<')
    print('Dados encontrados e enviados via Telegram'.upper())

# MONITORAMENTO
pd.options.mode.chained_assignment = None
resumoCloseGeral['flag'] = ''
resumoCloseGeral['sinal'] = ''

for i in range (1, len(resumo)):
    if resumoCloseGeral['MME 9'][i] < resumoCloseGeral['MME 3'][i]:
        resumoCloseGeral['flag'][i] = 'COMPRA'

    if resumoCloseGeral['MME 9'][i] > resumoCloseGeral['MME 3'][i]:
        resumoCloseGeral['flag'][i] = 'VENDA'


for x in range(1,len(resumoCloseGeral)):
    if resumoCloseGeral['flag'][x] == resumoCloseGeral['flag'][x-1]:
        resumoCloseGeral['sinal'][x] = ''
    else:
        resumoCloseGeral['sinal'][x] = 'sinal'

#flag = resumo['flag'].iloc[-1]

if resumoCloseGeral['sinal'].iloc[-1] == 'sinal':
    if resumoCloseGeral['flag'] == 'COMPRA':
        #enviaAlertaCompra()
        print('Compra')
    else:
        resumoCloseGeral['flag'][i] = 'VENDA'
        print('Venda')
        #enviaAlertaVenda()

resumoCloseGeral.tail(5)